# Using LDA to analyse legisalative text

In [1]:
import PyPDF2

Extract and clean the text from the pdf file. We will use the Residential Tenancy Act as the legislative text

In [6]:
file = open('residential.pdf', 'rb')
fileReader = PyPDF2.PdfFileReader(file)
number_of_pages = fileReader.getNumPages()

print('number of pages:', number_of_pages)

number of pages: 131


In [13]:
page = fileReader.getPage(0)
page_content = page.extractText()

print(page_content)

This version of the legislation is compiled and maintained 
in a database of legislation by the Parliamentary Counsel™s
Office and published on the NSW legislation website.
New South WalesResidential Tenancies Act 2010 No42
Status informationCurrency of version
Historical version for 6 January 2012 to 5 July 2012 (generated 12 July 2012 at 11:13). 
Legislation on the NSW legislation website is usually updated within 3 working days.
Provisions in force
All the provisions displayed in this version of the legislation have commenced. For 
commencement and other details
 see the Historical notes.
Does not include amendments by:
Statute Law (Miscellaneous Provisions) Act 2012 No 42 (not commenced Š to commence 
on 6.7.2012)
See also:
Residential Tenancies Amendment (Occupancy Agreements) Bill 2011
Community Housing Providers (Adoption of National Law) Bill 2012



In [11]:
# put the whole document in a list
def extract_data(pages, fileReader):
    page_text = []
    
    for i in range(pages):
        page = fileReader.getPage(i)
        page_content = page.extractText()
        page_text.append(page_content)
        
    return page_text

data = extract_data(number_of_pages, fileReader)
    
print(data[0])
print(len(data))

This version of the legislation is compiled and maintained 
in a database of legislation by the Parliamentary Counsel™s
Office and published on the NSW legislation website.
New South WalesResidential Tenancies Act 2010 No42
Status informationCurrency of version
Historical version for 6 January 2012 to 5 July 2012 (generated 12 July 2012 at 11:13). 
Legislation on the NSW legislation website is usually updated within 3 working days.
Provisions in force
All the provisions displayed in this version of the legislation have commenced. For 
commencement and other details
 see the Historical notes.
Does not include amendments by:
Statute Law (Miscellaneous Provisions) Act 2012 No 42 (not commenced Š to commence 
on 6.7.2012)
See also:
Residential Tenancies Amendment (Occupancy Agreements) Bill 2011
Community Housing Providers (Adoption of National Law) Bill 2012

131


### Applying the LDA algorithm, implementation from sci-kit learn

In [12]:
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer

In [14]:
NUM_TOPICS = 12

vectorizer = CountVectorizer(min_df=5, max_df=0.9, 
                             stop_words='english', lowercase=True, 
                             token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(data)

print(data_vectorized[0])

  (0, 177)	1
  (0, 23)	1
  (0, 232)	1
  (0, 203)	2
  (0, 181)	1
  (0, 124)	1
  (0, 243)	1
  (0, 113)	1
  (0, 74)	1
  (0, 73)	2
  (0, 158)	1
  (0, 312)	3
  (0, 101)	1
  (0, 372)	1
  (0, 241)	1
  (0, 261)	1
  (0, 98)	1


In [15]:
lda_model = LatentDirichletAllocation(n_topics=NUM_TOPICS, max_iter=10, learning_method='online')
lda_Z = lda_model.fit_transform(data_vectorized)
print(lda_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS)

(131, 12)


In [16]:
def print_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])
 
print("LDA Model:")
print_topics(lda_model, vectorizer)
print("=" * 20)

LDA Model:
Topic 0:
[('account', 0.5148762479599001), ('bond', 0.34035789616743756), ('rental', 0.3375389609132897), ('division', 0.31924135523232616), ('director-general', 0.2863052785027784), ('accommodation', 0.27935096200997017), ('tribunal', 0.27456957839607715), ('powers', 0.25689589559370696), ('proceedings', 0.25519865580885887), ('board', 0.24770925108948907)]
Topic 1:
[('person', 30.737007307467316), ('landlord', 29.569870890836263), ('goods', 25.19525295141994), ('section', 25.14761182602638), ('bond', 18.181813237348653), ('paid', 14.133165129852793), ('documents', 13.771197264040975), ('offence', 13.368142974977701), ('penalty', 13.138750240630417), ('tenant', 12.713426590142564)]
Topic 2:
[('tribunal', 31.210229156875837), ('rental', 31.165442035191273), ('bond', 27.24196628666371), ('order', 24.15993497872607), ('director-general', 22.513160284434036), ('proceedings', 19.392553361377516), ('claim', 15.57346242494831), ('person', 14.572283284242435), ('tenant', 13.4674696

### Visualization with 12 topics

In [17]:
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
panel

of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False
 [_prepare.py:257]


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
7       78.413048   20.647453       1        1  29.621688
8       63.877598 -175.178131       2        1  28.625848
4      -65.966789 -145.392288       3        1  13.668850
1      182.836914   53.636707       4        1  11.014749
2      -45.979820  172.587708       5        1   9.370971
11     153.041885  -76.206688       6        1   4.746427
10     -60.520874  -23.240561       7        1   1.462970
5     -164.946014  -56.223564       8        1   0.635617
9       83.858475  142.798859       9        1   0.619613
6     -135.147095   73.619225      10        1   0.077756
0      -12.994839   68.161552      11        1   0.077755
3       30.886906  -70.754616      12        1   0.077755, topic_info=     Category        Freq              Term       Total  loglift  logprob
term                                                                     
99    Default   69.000000              date   69.000000  30.0000  30.0000
390   Default  488.000000           tenancy  488.000000  29.0000  29.0000
398   Default  279.000000       termination  279.000000  28.0000  28.0000
287   Default  222.000000            person  222.000000  27.0000  27.0000
244   Default  291.000000            notice  291.000000  26.0000  26.0000
200   Default  582.000000          landlord  582.000000  25.0000  25.0000
74    Default   62.000000      commencement   62.000000  24.0000  24.0000
362   Default   38.000000               sec   38.000000  23.0000  23.0000
408   Default  239.000000          tribunal  239.000000  22.0000  22.0000
165   Default  132.000000             given  132.000000  21.0000  21.0000
188   Default   59.000000       information   59.000000  20.0000  20.0000
280   Default   80.000000           penalty   80.000000  19.0000  19.0000
391   Default  612.000000            tenant  612.000000  18.0000  18.0000
339   Default  100.000000            rental  100.000000  17.0000  17.0000
169   Default   71.000000             goods   71.000000  16.0000  16.0000
51    Default  103.000000              bond  103.000000  15.0000  15.0000
338   Default  201.000000              rent  201.000000  14.0000  14.0000
147   Default   58.000000          existing   58.000000  13.0000  13.0000
370   Default  103.000000            social  103.000000  12.0000  12.0000
98    Default   61.000000          database   61.000000  11.0000  11.0000
177   Default  119.000000           housing  119.000000  10.0000  10.0000
16    Default  125.000000             agent  125.000000   9.0000   9.0000
124   Default   88.000000              does   88.000000   8.0000   8.0000
363   Default  362.000000           section  362.000000   7.0000   7.0000
118   Default   69.000000  director-general   69.000000   6.0000   6.0000
157   Default   70.000000         following   70.000000   5.0000   5.0000
22    Default  515.000000         agreement  515.000000   4.0000   4.0000
393   Default  176.000000              term  176.000000   3.0000   3.0000
266   Default  217.000000             order  217.000000   2.0000   2.0000
328   Default   94.000000       regulations   94.000000   1.0000   1.0000
...       ...         ...               ...         ...      ...      ...
146   Topic12    0.027896          exercise   11.851604   1.1076  -5.9472
189   Topic12    0.026073               ing    8.350439   1.3902  -6.0149
109   Topic12    0.029799        department   19.902561   0.6552  -5.8812
283   Topic12    0.026515           periods    9.713858   1.2558  -5.9980
339   Topic12    0.035881            rental  100.144594  -0.7748  -5.6955
51    Topic12    0.036018              bond  103.558710  -0.8045  -5.6917
297   Topic12    0.027146              prem   11.807313   1.0841  -5.9745
350   Topic12    0.027057         residence   11.613475   1.0974  -5.9778
110   Topic12    0.027385           deposit   12.983715   0.9979  -5.9658
315   Topic12    0.027543           p

### What happens if we have fewer topics?

In [18]:
NUM_TOPICS = 5

lda_model = LatentDirichletAllocation(n_topics=NUM_TOPICS, max_iter=10, learning_method='online')
lda_Z = lda_model.fit_transform(data_vectorized)
print(lda_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS)

(131, 5)


In [19]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
panel

of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False
 [_prepare.py:257]


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
0      317.203064  -16.431667       1        1  36.275293
4       85.296890   58.230236       2        1  31.755560
1      -31.640680 -248.851624       3        1  24.069754
3     -144.242661  -23.368752       4        1   7.144293
2      211.244812 -245.103516       5        1   0.755101, topic_info=     Category        Freq          Term       Total  loglift  logprob
term                                                                 
99    Default   73.000000          date   73.000000  30.0000  30.0000
390   Default  478.000000       tenancy  478.000000  29.0000  29.0000
362   Default   46.000000           sec   46.000000  28.0000  28.0000
74    Default   59.000000  commencement   59.000000  27.0000  27.0000
157   Default   70.000000     following   70.000000  26.0000  26.0000
147   Default   53.000000      existing   53.000000  25.0000  25.0000
124   Default   81.000000          does   81.000000  24.0000  24.0000
398   Default  270.000000   termination  270.000000  23.0000  23.0000
234   Default   41.000000        months   41.000000  22.0000  22.0000
361   Default   38.000000      schedule   38.000000  21.0000  21.0000
279   Default   73.000000       payment   73.000000  20.0000  20.0000
70    Default   34.000000        clause   34.000000  19.0000  19.0000
98    Default   64.000000      database   64.000000  18.0000  18.0000
22    Default  514.000000     agreement  514.000000  17.0000  17.0000
169   Default   68.000000         goods   68.000000  16.0000  16.0000
203   Default   18.000000           law   18.000000  15.0000  15.0000
312   Default   18.000000    provisions   18.000000  14.0000  14.0000
415   Default   29.000000          used   29.000000  13.0000  13.0000
65    Default   34.000000       charges   34.000000  12.0000  12.0000
422   Default   28.000000         water   28.000000  11.0000  11.0000
33    Default   60.000000         apply   60.000000  10.0000  10.0000
50    Default   68.000000         board   68.000000   9.0000   9.0000
244   Default  275.000000        notice  275.000000   8.0000   8.0000
328   Default   99.000000   regulations   99.000000   7.0000   7.0000
380   Default   39.000000       subject   39.000000   6.0000   6.0000
311   Default   43.000000     provision   43.000000   5.0000   5.0000
156   Default   99.000000         fixed   99.000000   4.0000   4.0000
352   Default   37.000000       respect   37.000000   3.0000   3.0000
363   Default  365.000000       section  365.000000   2.0000   2.0000
339   Default   89.000000        rental   89.000000   1.0000   1.0000
...       ...         ...           ...         ...      ...      ...
85     Topic5    0.033377    considered    8.270196  -0.6265  -8.0411
295    Topic5    0.024441   practicable    6.152255  -0.6422  -8.3528
409    Topic5    0.025430          unal    6.403584  -0.6426  -8.3131
371    Topic5    0.031378          soon    7.919545  -0.6449  -8.1029
10     Topic5    0.024554       actions    6.216072  -0.6480  -8.3482
72     Topic5    0.025692    collection    6.506215  -0.6483  -8.3029
197    Topic5    0.025531          kind    6.501358  -0.6538  -8.3091
38     Topic5    0.026223      approval    6.738467  -0.6629  -8.2824
413    Topic5    0.029556         usage    7.951116  -0.7087  -8.1627
34     Topic5    0.050395     appointed   16.694356  -0.9169  -7.6291
229    Topic5    0.109447        member   49.302564  -1.2242  -6.8536
292    Topic5    0.030550          post    8.334266  -0.7227  -8.1297
420    Topic5    0.043386         wales   14.013767  -0.8916  -7.7789
289    Topic5    0.029053    personally    7.953383  -0.7262  -8.1799
241    Topic5    0.058886           new   27.146645  -1.2473  -7.4734
231    Topic5    0.053336      minister   24.979238  -1.2631  -7.5724
320    Topic5    0.029932        rebate    9.121973  -0.8334  -8.1501
309    Topic5    0.037308      provider   16.486689

In [20]:
import numpy as np
import mglearn

In [22]:
sorting = np.argsort(lda_model.components_)[:, ::-1]
features = np.array(vectorizer.get_feature_names())

In [24]:
mglearn.tools.print_topics(topics=range(5), feature_names=features, sorting=sorting, topics_per_chunk=5, n_words=20)

topic 0       topic 1       topic 2       topic 3       topic 4       
--------      --------      --------      --------      --------      
agreement     landlord      sec           tenancy       landlord      
tenancy       notice        date          existing      tenant        
tenant        person        commencement  commencement  premises      
termination   tenant        provisions    does          agreement     
premises      section       law           months        tenancy       
landlord      premises      miscellaneous schedule      person        
section       tribunal      notes         clause        section       
notice        goods         amended       charges       rent          
term          board         used          agreement     agent         
order         rent          following     water         information   
tribunal      order         office        section       penalty       
housing       given         member        payment       paid          
fixed 

In [32]:
filters = [('agreement', ''), ('tenancy', ''), ('tenant', ''), 
           ('landlord', ''), ('section', ''), ('premises', ''), ('provision', ''), ('person', ''), ('tribunal', '')]

for k, v in filters:
    data = [d.replace(k, v) for d in data]

In [37]:
NUM_TOPICS = 5

data_vectorized = vectorizer.fit_transform(data)

lda_model = LatentDirichletAllocation(n_topics=NUM_TOPICS, max_iter=10, learning_method='online')
lda_Z = lda_model.fit_transform(data_vectorized)

pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
panel


of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False
 [_prepare.py:257]


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
0       69.921196 -119.985168       1        1  33.077500
4      -57.002979 -149.621735       2        1  26.322734
3       38.946796   53.006046       3        1  21.960357
1      -71.457985  -15.557214       4        1   9.540706
2      152.738190  -19.336967       5        1   9.098703, topic_info=     Category        Freq           Term       Total  loglift  logprob
term                                                                  
354   Default  221.000000            sec  221.000000  30.0000  30.0000
390   Default  254.000000    termination  254.000000  29.0000  29.0000
74    Default   82.000000   commencement   82.000000  28.0000  28.0000
330   Default  190.000000           rent  190.000000  27.0000  27.0000
188   Default   65.000000    information   65.000000  26.0000  26.0000
99    Default  100.000000           date  100.000000  25.0000  25.0000
98    Default   60.000000       database   60.000000  24.0000  24.0000
177   Default  107.000000        housing  107.000000  23.0000  23.0000
362   Default   93.000000         social   93.000000  22.0000  22.0000
202   Default   42.000000            law   42.000000  21.0000  21.0000
147   Default   45.000000       existing   45.000000  20.0000  20.0000
16    Default  128.000000          agent  128.000000  19.0000  19.0000
231   Default   36.000000  miscellaneous   36.000000  18.0000  18.0000
243   Default  274.000000         notice  274.000000  17.0000  17.0000
385   Default  167.000000           term  167.000000  16.0000  16.0000
265   Default  219.000000          order  219.000000  15.0000  15.0000
227   Default   47.000000          means   47.000000  14.0000  14.0000
51    Default   40.000000           bond   40.000000  13.0000  13.0000
353   Default   29.000000       schedule   29.000000  12.0000  12.0000
124   Default   75.000000           does   75.000000  11.0000  11.0000
331   Default   42.000000         rental   42.000000  10.0000  10.0000
70    Default   26.000000         clause   26.000000   9.0000   9.0000
407   Default   39.000000           used   39.000000   8.0000   8.0000
156   Default   93.000000          fixed   93.000000   7.0000   7.0000
279   Default   80.000000        penalty   80.000000   6.0000   6.0000
233   Default   38.000000         months   38.000000   5.0000   5.0000
242   Default   25.000000          notes   25.000000   4.0000   4.0000
170   Default   53.000000         ground   53.000000   3.0000   3.0000
25    Default   26.000000        amended   26.000000   2.0000   2.0000
191   Default   27.000000   investigator   27.000000   1.0000   1.0000
...       ...         ...            ...         ...      ...      ...
306    Topic5    2.718659          purpo    6.495060   1.5261  -5.8436
415    Topic5    2.779933            way    6.926411   1.4841  -5.8213
382    Topic5    2.942045        tenancy    7.562077   1.4530  -5.7646
161    Topic5    2.397882       function    6.608429   1.3833  -5.9691
192    Topic5    3.698611            ion   10.283012   1.3745  -5.5358
329    Topic5    4.410296        removed   12.476630   1.3571  -5.3598
248    Topic5    5.125044     obligation   14.510716   1.3563  -5.2096
348    Topic5   11.367802          right   33.005844   1.3311  -4.4129
337    Topic5    6.370809        require   18.604224   1.3254  -4.9920
42     Topic5    4.073704          ation   12.065058   1.3113  -5.4392
107    Topic5    4.473946    definitions   13.414902   1.2989  -5.3455
342    Topic5    3.944895      residence   11.881556   1.2945  -5.4713
141    Topic5    6.711109       evidence   20.512331   1.2798  -4.9400
123    Topic5    9.754812      documents   36.362791   1.0813  -4.5660
16     Topic5   24.456408          agent  128.659184   0.7368  -3.6468
418    Topic5    7.280738        writing   26.817213   1.0932  -4.8585
331    Topic5    9.117068         rental   42.396632   0.8601  -4.6336
5

In [39]:
sorting = np.argsort(lda_model.components_)[:, ::-1]
features = np.array(vectorizer.get_feature_names())
mglearn.tools.print_topics(topics=range(5), feature_names=features, sorting=sorting, topics_per_chunk=5, n_words=35)

topic 0       topic 1       topic 2       topic 3       topic 4       
--------      --------      --------      --------      --------      
order         sec           information   rent          termination   
tribunal      commencement  database      term          notice        
possession    date          means         penalty       housing       
notice        existing      agent         paid          social        
agent         law           investigator  payable       term          
make          miscellaneous includes      agent         fixed         
proceedings   schedule      division      terms         given         
application   does          listing       notice        ground        
goods         clause        operator      consent       tribunal      
director-generalbond          does          tribunal      end           
given         months        right         regulations   division      
regulations   used          penalty       section       charges       
divi